### AutoGen Intro Tutorial: Travel Planning

This is a sample solution for the AutoGen Intro tutorial. Keep in mind that there are different ways to solve the tasks!

### Getting started

First, we need to import the necessary modules.

In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

Before defining the agents that will be used for the travel planning, we first need to set up the model client given our Azure credentials that we load from our env file.

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=deployment,
    model="gpt-4o",
    api_version="2024-08-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_KEY
)

We will use the class AssistantAgent by AutoGen which is a built in language model that has the ability to use tools, we will define a very simple tool called get_weather to showcase this feature.
You can also implement your own tool which one of your agents could then use.

In [7]:
def get_weather(location: str, date:str) -> str:
    return f"The weather in {location} on {date} is sunny, 24°C."

In [ ]:
def get_price(activities: str) -> str:
    activity_prices = {
        "museum": "10–15 EUR",
        "boat tour": "25–40 EUR",
        "cooking class": "35–60 EUR",
        "bike rental": "10–20 EUR",
        "walking tour": "15–25 EUR",
        "wine tasting": "30–50 EUR"
    }

    activities = activities.lower().split(",")
    results = []

    for activity in activities:
        activity = activity.strip()
        price = activity_prices.get(activity, "20–40 EUR")  # default fallback
        results.append(f"{activity.title()}: approx. {price}")

    return "\n".join(results)

Now, we can define the agents.

In [9]:
planner_agent = AssistantAgent(
    "planner_agent",
    model_client=model_client,
    description="A helpful assistant that can plan trips.",
    system_message="You are a helpful assistant that can suggest a travel plan for a user based on their request.",
)

local_agent = AssistantAgent(
    "local_agent",
    model_client=model_client,
    description="A local assistant that can suggest local activities or places to visit.",
    tools=[get_price],
    system_message="You are a helpful assistant that can suggest authentic and interesting local activities or places to visit for a user and can utilize any context information provided. You can also provide information about the prices of activities.",
)

language_agent = AssistantAgent(
    "language_agent",
    model_client=model_client,
    description="A helpful assistant that can provide language tips for a given destination.",
    system_message="You are a helpful assistant that can review travel plans, providing feedback on important/critical tips about how best to address language or communication challenges for the given destination. If the plan already includes language tips, you can mention that the plan is satisfactory, with rationale.",
)

weather_agent = AssistantAgent(
    "weather_agent",
    model_client=model_client,
    description="A helpful assistant that can provide weather information for a given location and date.",
    tools=[get_weather],
    system_message="You are a helpful assistant that can provide weather information for a given location and date. You can also utilize any context information provided.",
)

travel_summary_agent = AssistantAgent(
    "travel_summary_agent",
    model_client=model_client,
    description="A helpful assistant that can summarize the travel plan.",
    system_message="You are a helpful assistant that should summarize the content gathered by the planner, local, language and weather agent.",
)

Next, we need to create the team of the agents. We will use the simple yet effective RoundRobinChat a team configuration where all agents share the same context and take turns responding in a round-robin fashion. 
Each agent, during its turn, broadcasts its response to all other agents, ensuring that the entire team maintains a consistent context.

In [ ]:
termination = TextMentionTermination("TERMINATE")
group_chat = RoundRobinGroupChat(
    [planner_agent, local_agent, language_agent, travel_summary_agent], termination_condition=termination
)

task = "I want to plan a 3-day trip to Rome, Italy. Suggest activities and the approximate price for that."    

#here we run the agent chat and stream the conversation in the console
await Console(group_chat.run_stream(task=task))

await model_client.close()


---------- TextMessage (user) ----------
I want to plan a 3-day trip to Rome, Italy. Suggest activities and the approximate price for that.
---------- TextMessage (planner_agent) ----------
Certainly! Rome is a magnificent city steeped in history and culture, and three days is just enough to scratch its surface. Below is a suggested itinerary for your trip. Keep in mind that actual prices may vary based on your preferences and the time of year you travel (peak or off-season). I'll provide approximate costs in Euros (€).

---

### **Day 1: Ancient Rome & Monti District**

**Morning**  
1. **Colosseum and Roman Forum**  
   - Explore the iconic Colosseum, a symbol of Ancient Rome, and the adjacent Roman Forum, once the center of Roman life.  
   - Approx. Cost: €18 (combined Colosseum, Roman Forum, and Palatine Hill ticket)  
   - Time: 3–4 hours  

2. **Palatine Hill**  
   - After the Colosseum, visit Palatine Hill. From here, you get stunning views of the Roman Forum below.  
   - Inc

c:\Users\laetitiamaar\AppData\Local\Programs\Python\Python313\Lib\site-packages\autogen_agentchat\agents\_assistant_agent.py:955: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  model_result = await model_client.create(


---------- ToolCallRequestEvent (local_agent) ----------
[FunctionCall(id='call_WhPXC3OzyNW8P6Cj7hEDvfqi', arguments='{"activities": "Colosseum and Roman Forum ticket"}', name='get_price'), FunctionCall(id='call_TqXlc7xyxmjvbqIN41ngRh42', arguments='{"activities": "Capitoline Museums ticket"}', name='get_price'), FunctionCall(id='call_5jEq4HNA7yOb0MATig8SX9jG', arguments='{"activities": "Vatican Museums and Sistine Chapel ticket"}', name='get_price'), FunctionCall(id='call_i6mzLoOpATZrm41l5Px7pZDz', arguments='{"activities": "Castel Sant’Angelo ticket"}', name='get_price'), FunctionCall(id='call_nZvT5tBgIfG6sSDuTXIQltgW', arguments='{"activities": "Borghese Gallery ticket"}', name='get_price')]
---------- ToolCallExecutionEvent (local_agent) ----------
[FunctionExecutionResult(content='Colosseum And Roman Forum Ticket: approx. 20–40 EUR', name='get_price', call_id='call_WhPXC3OzyNW8P6Cj7hEDvfqi', is_error=False), FunctionExecutionResult(content='Capitoline Museums Ticket: approx. 20–40